# Preprocessing Notebook: `1_df_studies.ipynb`

### Description
Processes the `studies` table from the AACT Clinical Trials database to create a cleaned and feature-engineered dataset ready for downstream analysis and modeling.  
This is the **first notebook** in the data preprocessing pipeline.

### Key Steps
- Load selected columns from the `studies` table via SQLAlchemy.  
- Filter only *interventional* studies.  
- Generate binary outcome variable (`overall_status` → success/failure).  
- Handle missing values and convert column datatypes.  
- Parse start and completion dates to calculate **study duration**.  
- Clean and standardize the `phase` column.  
- Impute missing `enrollment` values and **reassign mixed phases (Phase 1/2, Phase 2/3)** based on median enrollment size.  
- One-hot encode `phase` and encode flag like columns 
- Save cleaned dataset to: `../data/processed/studies_clean.csv`

### Output
- ✅ `studies_clean.csv` — Cleaned and standardized dataset used for merges in subsequent notebooks.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os 

load_dotenv()
DATABASE_URL = os.getenv('DATABASE_URL')
engine = create_engine(DATABASE_URL)

In [2]:
# Load selected fields from the `studies` table
query = '''
SELECT
    nct_id,
    start_month_year,
    completion_month_year,
    study_type,
    enrollment,
    overall_status,
    phase,
    number_of_arms,
    has_dmc,
    has_expanded_access,
    is_fda_regulated_drug,
    is_fda_regulated_device
FROM ctgov.studies;
'''
df_studies = pd.read_sql(query, engine)

In [3]:
# Filter for interventional studies and map outcomes to binary target
df_studies = df_studies[df_studies['study_type'] == 'INTERVENTIONAL'].copy()

final_outcomes = ['COMPLETED', 'TERMINATED', 'WITHDRAWN']
df_studies = df_studies[df_studies['overall_status'].isin(final_outcomes)].copy()

df_studies['overall_status'] = df_studies['overall_status'].map({
    'COMPLETED': 1,
    'TERMINATED': 0,
    'WITHDRAWN': 0
})

In [4]:
# Fill missing values for key categorical columns
df_studies['is_fda_regulated_drug'] = df_studies['is_fda_regulated_drug'].fillna('unknown')
df_studies['is_fda_regulated_device'] = df_studies['is_fda_regulated_device'].fillna('unknown')
df_studies['has_expanded_access'] = df_studies['has_expanded_access'].fillna('unknown')
df_studies['has_dmc'] = df_studies['has_dmc'].fillna('unknown')

In [5]:
# Convert numeric columns and impute missing values
df_studies['number_of_arms'] = pd.to_numeric(df_studies['number_of_arms'], errors = 'coerce')
df_studies['number_of_arms'] = df_studies['number_of_arms'].fillna(df_studies['number_of_arms'].median())

# Drop rows with missing dates
df_studies = df_studies.dropna(subset = ['start_month_year', 'completion_month_year'])

In [6]:
# Parse date columns and compute study duration
from dateutil import parser
def parse_date_safe(val):
    try:
        return parser.parse(val)
    except:
        return pd.NaT

df_studies['start_month_year'] = df_studies['start_month_year'].apply(parse_date_safe)
df_studies['completion_month_year'] = df_studies['completion_month_year'].apply(parse_date_safe)

df_studies['duration_of_study'] = (
    df_studies['completion_month_year'] - df_studies['start_month_year']
).dt.days

In [7]:
# Clean and standardise `phase` column
df_studies['phase'] = df_studies['phase'].replace({'NA': 'NOT APPLICABLE', 'None': 'NOT APPLICABLE'})
df_studies['phase'] = df_studies['phase'].fillna('NOT APPLICABLE')
df_studies['phase'] = df_studies['phase'].str.lower()

# Impute enrollment by median per phase
df_studies['enrollment'] = pd.to_numeric(df_studies['enrollment'], errors = 'coerce')
df_studies['enrollment'] = df_studies.groupby('phase')['enrollment'].transform(lambda x: x.fillna(x.median()))

# Reassign mixed phases based on enrollment size
size = 50
df_studies.loc[(df_studies['phase'] == 'phase1/phase2') & (df_studies['enrollment'] <= size), 'phase'] = 'phase1'
df_studies.loc[(df_studies['phase'] == 'phase1/phase2') & (df_studies['enrollment'] > size), 'phase'] = 'phase2'

median_size = (
    df_studies[df_studies['phase'] == 'phase2']['enrollment'].median() +
    df_studies[df_studies['phase'] == 'phase3']['enrollment'].median()
) / 2

df_studies.loc[(df_studies['phase'] == 'phase2/phase3') & (df_studies['enrollment'] <= median_size), 'phase'] = 'phase2'
df_studies.loc[(df_studies['phase'] == 'phase2/phase3') & (df_studies['enrollment'] > median_size), 'phase'] = 'phase3'

df_studies['phase'] = df_studies['phase'].replace({'early_phase1': 'phase1'})

In [8]:
# Encode flag like cols and one-hot encode `phase`
binary_columns = ['has_dmc', 'is_fda_regulated_device', 'is_fda_regulated_drug', 'has_expanded_access']
for col in binary_columns:
    df_studies[col] = df_studies[col].map({True: 1, False: 0, 'unknown': -1})

# Simplify and one-hot encode phase
df_studies['phase'] = df_studies['phase'].str.replace('phase', '', regex=False)
df_studies = pd.get_dummies(df_studies, columns = ['phase'], prefix = 'phase', dtype = int)

In [9]:
# Drop unused columns
df_studies = df_studies.drop(columns = ['start_month_year', 'completion_month_year'])

In [10]:
# Save cleaned dataset to CSV
df_studies.to_csv('../data/processed/studies_clean.csv', index=False)

---

## Summary  
This notebook successfully cleaned and standardized the `studies` table, preparing it for downstream feature integration and modeling.  

Key transformations included:  
- Parsing and computing **study duration** from start and completion dates.  
- Cleaning and **reassigning mixed study phases** based on enrollment characteristics.  
- Handling missing data and creating **binary target** and **flag variables**.  

The resulting dataset — `studies_clean.csv` — forms the foundation for all subsequent merges and analyses.

---

📂 **Next Notebook:** `2_df_baseline_features.ipynb` → Adds demographic and baseline characteristics for each trial.
